In [2]:
import json
import pandas as pd
import requests
from turfpy.transformation import intersect, union
from geojson import FeatureCollection

In [4]:
url_1 = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/refs/heads/master/departements/51-marne/departement-51-marne.geojson"
url_2 = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/refs/heads/master/departements/10-aube/departement-10-aube.geojson"
url_3 = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/refs/heads/master/departements/02-aisne/departement-02-aisne.geojson"
url_4 = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/refs/heads/master/departements/52-haute-marne/departement-52-haute-marne.geojson"
url_5 = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/refs/heads/master/departements/77-seine-et-marne/departement-77-seine-et-marne.geojson"
departements = []
for url in [url_1, url_2, url_3, url_4, url_5]:
  res = requests.get(url)
  departements.append(res.json())
geojson = FeatureCollection(departements)


In [7]:
with open('../departements.geojson', "w") as file:
  json.dump(geojson, file)

In [22]:
df = pd.read_csv('../communes_aoc_champagne.csv', sep=';')
df["CI"] = (
        df["CI"].astype(str).map(lambda x: (5 - len(x)) * "0" + x)
    )


In [ ]:
communes = []
for c in df.CI.tolist():
  url = f"https://cadastre.data.gouv.fr/bundler/cadastre-etalab/communes/{c}/geojson/communes"
  try:
    res = requests.get(url)
    communes += res.json()['features']
  except:
    print(c)


02161
02669
51063
51064
51331
51347
51411
51637
51651
52018


In [25]:
with open('../communes_production.geojson', "w") as file:
  json.dump(FeatureCollection(communes), file)

In [1]:
import geopandas as gpd

# Charger le fichier GeoJSON
gdf = gpd.read_file("../communes_production.geojson")

# Simplifier chaque géométrie en fonction d'une tolérance (en unités CRS)
gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.0001, preserve_topology=True)

# Sauvegarder le fichier allégé
gdf.to_file("../communes_production_light.geojson", driver='GeoJSON')

In [1]:
import geopandas as gpd

# Charger les fichiers GeoJSON
geojson1 = gpd.read_file("../vineyard.geojson")
geojson2 = gpd.read_file("../communes_production.geojson")

In [2]:
intersection = gpd.overlay(geojson1, geojson2, how='intersection')

# Sauvegarder le résultat
intersection.to_file("../communes_aoc.geojson", driver="GeoJSON")

In [3]:
with open("../communes_production_light.geojson", "r") as file:
  communes = json.load(file)

In [4]:
input_file = "../communes_production_light.geojson"
output_dir = "../communes"

In [12]:
gdf = gpd.read_file(input_file)

In [14]:
gdf['departement'] = gdf['id'].str[:2]

In [16]:
departements = gdf['departement'].unique()
departements

array(['02', '10', '51', '52', '77'], dtype=object)

In [ ]:
import os
for dep in departements:

    gdf_dep = gdf[gdf['departement'] == dep]
    output_file = os.path.join(output_dir, f"{dep}.geojson")
    gdf_dep.to_file(output_file, driver="GeoJSON")
    print(f"Fichier créé : {output_file}")

Fichier créé : ../communes/02.geojson
Fichier créé : ../communes/10.geojson
Fichier créé : ../communes/51.geojson
Fichier créé : ../communes/52.geojson
Fichier créé : ../communes/77.geojson


In [10]:
import json
from geojson import FeatureCollection

In [11]:
with open("../communes_aoc_light.geojson", "r") as file:
  geojson = json.load(file)

In [17]:
for f in geojson['features']:
  data = FeatureCollection([f])
  with open(f"../communes/aoc/{f['properties']['id']}.geojson", "w") as file:
    json.dump(data, file)